In [9]:
import polars as pl
import trendline
import crypto_data
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
data = crypto_data.download_crypto_ohlc_data(symbols=['SOLO/USDT'])

Initializing data download for: bybit, Timeframe: 1d, Periods: 365...
Data for SOLO/USDT:
  Start: 2024-01-14 00:00:00
  End: 2025-01-12 00:00:00
Last row: 
Timestamp    2025-01-12 00:00:00
Open                      0.5139
High                     0.52303
Low                      0.49384
Close                    0.50666
Volume                 568464.98
Name: 364, dtype: object

  Expected Interval: 1d, Actual Interval: 1 days 00:00:00
Finished downloading data for 1 symbols


In [3]:
data

SOLO/USDT                                                
     Timestamp     Open     High      Low    Close      Volume
0   2024-01-14  0.17180  0.17410  0.16425  0.16900   932131.05
1   2024-01-15  0.16900  0.17450  0.16680  0.16850   570634.87
2   2024-01-16  0.16850  0.17117  0.16664  0.16830   511738.94
3   2024-01-17  0.16830  0.16830  0.16172  0.16303   310708.94
4   2024-01-18  0.16303  0.16840  0.15704  0.15859   355657.75
..         ...      ...      ...      ...      ...         ...
360 2025-01-08  0.45570  0.48958  0.44735  0.48516  1767230.83
361 2025-01-09  0.48516  0.49548  0.45163  0.45953   717948.86
362 2025-01-10  0.45953  0.48123  0.45093  0.46103   634382.41
363 2025-01-11  0.46103  0.52439  0.45692  0.51390  1275263.67
364 2025-01-12  0.51390  0.52303  0.49384  0.50666   568464.98

[365 rows x 6 columns]

In [18]:
data = data['SOLO/USDT']

A ver ideas, para el patrón que busco puedo aplicar los siguientes condicionales: 
1. price above 21 ema
2. slope of trendline < 0
3. lenght de 3 a 15 candles? 

In [64]:
def trendline_breakout(close: np.array, lookback: int):
    # Initializing arrays for support/resistance and signals
    s_tl = np.zeros(len(close))
    s_tl[:] = np.nan

    r_tl = np.zeros(len(close))
    r_tl[:] = np.nan

    signals = np.zeros(len(close))

    for i in range(lookback, len(close)):
        # We don't include the current candle, so to get the breakout
        window = close[i - lookback:i]

        s_coefs, r_coefs = trendline.fit_trendlines_single(window)

        # Find current value of line, projected forward to current bar
        s_val = s_coefs[1] + lookback * s_coefs[0]
        r_val = r_coefs[1] + lookback * r_coefs[0]

        s_tl[i] = s_val
        r_tl[i] = r_val

        if close[i] > r_val:
            signals[i] = 1.0
        elif close[i] < s_val:
            signals[i] = -1.0
        else:
            signals[i] = signals[i - 1]
    
    return s_tl, r_tl, signals


In [65]:
def plot_channel_with_breakout(data, open_prices, high_prices, low_prices, close_prices, lookback: int = 20):
    # Calculate trendlines using your original algorithm
    s_tl, r_tl, signals = trendline_breakout(close_prices.to_numpy(), lookback)
    
    # Create the main figure
    fig = go.Figure()
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=np.arange(len(close_prices)),
            open=open_prices,
            high=high_prices,
            low=low_prices,
            close=close_prices,
            increasing_line_color='#26A69A',  # Green candles
            decreasing_line_color='#EF5350',  # Red candles
            name='Price'
        )
    )
    
    # Add support trendline
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(s_tl)),
            y=s_tl,
            mode='lines',
            name='Support',
            line=dict(color='white', width=1, dash='dot'),
            showlegend=True
        )
    )
    
    # Add resistance trendline
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(r_tl)),
            y=r_tl,
            mode='lines',
            name='Resistance',
            line=dict(color='white', width=1, dash='dot'),
            showlegend=True
        )
    )
    
    # Add breakout points
    breakout_idx = np.where(np.abs(np.diff(signals)) > 0)[0] + 1
    breakout_values = close_prices[breakout_idx]
    
    # Add markers for breakout points
    fig.add_trace(
        go.Scatter(
            x=breakout_idx,
            y=breakout_values,
            mode='markers',
            name='Breakouts',
            marker=dict(
                size=10,
                symbol='triangle-up',
                color='yellow',
                line=dict(color='black', width=2)
            ),
            showlegend=True
        )
    )
    
    # Update layout for dark theme
    fig.update_layout(
        title='Trendline Channel with Breakouts',
        xaxis_rangeslider_visible=False,  # Hide rangeslider
        height=800,
        yaxis=dict(
            gridcolor='rgba(128,128,128,0.1)',
            zerolinecolor='rgba(128,128,128,0.1)',
        ),
        xaxis=dict(
            gridcolor='rgba(128,128,128,0.1)',
            zerolinecolor='rgba(128,128,128,0.1)',
        ),
        showlegend=True,
        legend=dict(
            bgcolor='rgba(0,0,0,0.5)',
            bordercolor='rgba(255,255,255,0.2)'
        )
    )
    
    return fig


fig = plot_channel_with_breakout(data,
                            open_prices=data['Open'], 
                            high_prices=data['High'],
                            low_prices=data['Low'],
                            close_prices=data['Close']
                            )

In [66]:
fig